# Done Soon? Data Analysis

## Import Libraries

In [ ]:
%matplotlib inline
import random
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import MaxAbsScaler
from sklearn.inspection import permutation_importance
from sklearn.metrics import f1_score
import pymongo
import plotly.express as px
import math
from enum import Enum
from tqdm.notebook import tqdm
import multiprocessing as mp

## Notebook Configuration

In [ ]:
class InstanceType(Enum):
    ALL = 1
    OPTIMIZATION = 2
    SAT = 3

data_types_to_consider = InstanceType.OPTIMIZATION

## Get Data - Mongodb

In [ ]:
def string_to_instance_type(s):
    return InstanceType.SAT if s == 'SAT' else InstanceType.OPTIMIZATION


conn_str = f"mongodb://localhost"
mongo_client = pymongo.MongoClient(conn_str, port=27017)

#all data
all_data = pd.DataFrame(mongo_client.done_soon.problems.find({}))
all_data = all_data[all_data['error'] !=True ]
all_data = all_data[all_data['problem_type'].map(string_to_instance_type) == data_types_to_consider ]

#manual filtering (2hour)
timeout_after_hour = all_data[all_data.time_to_solution > 7199 * 1000] #>2hours
finishes_after_lower_bound = all_data[all_data.time_to_solution > 72 * 1000] #>72seconds

#print result filtering / Sanity check
print(f"Unsolved: {len(timeout_after_hour)}")
print(f"Class distribution: {len(timeout_after_hour)/(len(timeout_after_hour)+len(finishes_after_lower_bound))}")

#number of values of len of stats
finishes_after_lower_bound.statistics.map(len).value_counts()

In [ ]:
length_of_stats = np.array([len(x) for x in all_data['statistics']])
time_to_solution = all_data['time_to_solution']
plt.scatter(length_of_stats, time_to_solution, marker='.')
plt.title("Suspicious data (shouldn't this be linear?)")
plt.xlabel("length of stats array")
plt.ylabel("time to solution")

In [ ]:
length_of_stats = np.array([len(x) for x in all_data['statistics']]) / 2
time_to_solution = all_data['time_to_solution'] / 72000
filtered_all_data = all_data[((all_data['time_to_solution'] / 72000) - length_of_stats) < 5]

plt.rcParams["figure.figsize"] = (10,6)
filtered_length_of_stats = np.array([len(x) for x in filtered_all_data['statistics']]) / 2
filtered_time_to_solution = filtered_all_data['time_to_solution'] / 72000

plt.scatter(length_of_stats, time_to_solution, marker='.', color='red', alpha=0.4)
plt.scatter(filtered_length_of_stats, filtered_time_to_solution, marker='.')
plt.plot(np.arange(0, 100, 1), np.arange(0, 100, 1), color='black', alpha=0.4)
plt.xlabel("length of stats array")
plt.ylabel("time to solution")
# plt.tight_layout()

In [ ]:
no_small_data = all_data[all_data['statistics'].map(len) > 5]
length_of_stats = np.array([len(x) for x in no_small_data['statistics']]) / 2
i = range(0, len(no_small_data.index))
plt.barh(i, np.sort(length_of_stats))

In [ ]:
# all_data = filtered_all_data
filtered_features_at_percent = {}

for i in tqdm(range(1,200)):
    df_i=[]
    for id, problem in filtered_all_data.iterrows(): 
        for index, p in enumerate(problem.statistics):
            if index == i:
                new_p = dict(p['features'])
                new_p=cleanup(new_p)
                if i!=1:
                    new_p=gradients(df_prev.loc[id], new_p)
                new_p['mzn'] = problem['mzn']
                new_p['dzn'] = problem['dzn']
                new_p['solved_within_time_limit'] = problem['time_to_solution'] < 7199 * 1000
                df_i.append((id, new_p))
    df_i = pd.DataFrame([a[1] for a in df_i], index=[a[0] for a in df_i])
    df_i=df_i.fillna(value = 0)
    if i!=0:   
        filtered_features_at_percent[i]=df_i
    df_prev=df_i

In [ ]:
len(filtered_all_data[filtered_all_data['time_to_solution'] > 3600 * 1000])

In [ ]:
filtered_features_at_percent[1]['solved_within_time_limit'].value_counts()

In [ ]:
# some dataframe df
all_data['time_to_solution'].sort_values().plot(use_index=False)

In [ ]:
sub_data=all_data[all_data['time_to_solution']>7200]
sub_data['time_to_solution'].sort_values().plot(use_index=False)
#plt.plot(all_data[all_data['time_to_solution']>6000])

In [ ]:
#test
#finishes_after_lower_bound.statistics[1]

In [ ]:
def cleanup(df):
    del df["decision_level_sat"]
    del df["ewma_decision_level_mip"]
    del df["decision_level_mip"]
#     del df["best_objective"]
#     df["unassnVar"]   = (2**df['vars']) - df['opennodes']
#     df["fracFailUnassn"]     = df['conflicts'] / df['unassnVar']         # num failures/ num open nodes
    df["fracOpenVisit"]  = (df['vars'] - df['opennodes']) / df['opennodes']       # ratio of open nodes to visited nodes (how much of soln space explored)
    df["fracBoolVars"]     = df['boolVars'] / df['vars']                 # num bools / total num of vars
    df["fracPropVars"]     = df['propagations'] / df['vars']        # num propagations/ total num of vars
#     df["frac_unassigned"] = df['unassnVar'] / df['vars']  # current assignments/ total vars
    df["fracLongClauses"] = df['long'] + df['bin'] + df['tern']         # fraction of learnt clauses that have more than 3 literals
    df["freqBackjumps"]  = df['back_jumps']/df['search_time']
    return df

In [ ]:
def gradients(df_prev, df_curr):
    keys=['conflicts','ewma_conflicts','decisions','search_iterations','opennodes','ewma_opennodes',
          'vars','back_jumps','ewma_back_jumps','solutions','total_time','search_time','intVars',
          'propagations','sat_propagations','ewma_propagations','propagators','boolVars','learnt',
          'bin','tern','long','peak_depth','decision_level_engine','ewma_decision_level_engine',
          'decision_level_treesize','clause_mem','prop_mem','ewma_best_objective',
          'fracOpenVisit','fracBoolVars','fracPropVars','freqBackjumps', 'best_objective']
    for i in keys:
        df_curr[i+'_gradient']=(df_curr[i]-df_prev[i])/0.05*7200
    return df_curr

In [ ]:
len(all_data[all_data['time_to_solution'] > 7199 * 1000]), len(all_data[all_data['time_to_solution'] < 7199 * 1000]), len(all_data)

In [ ]:
features_at_percent = {}

for i in tqdm(range(1,200)):
    df_i=[]
    for id, problem in all_data.iterrows(): 
        for index, p in enumerate(problem.statistics):
            if index == i:
                new_p = dict(p['features'])
                new_p=cleanup(new_p)
                if i!=1:
                    new_p=gradients(df_prev.loc[id], new_p)
                new_p['mzn'] = problem['mzn']
                new_p['dzn'] = problem['dzn']
                new_p['solved_within_time_limit'] = problem['time_to_solution'] < 7199 * 1000 \
                or np.logical_not(np.isnan(problem['time_to_solution']))
                df_i.append((id, new_p))
    df_i = pd.DataFrame([a[1] for a in df_i], index=[a[0] for a in df_i])
    df_i=df_i.fillna(value = 0)
    if i!=0:   
        features_at_percent[i]=df_i
    df_prev=df_i

In [ ]:
nans=features_at_percent[20].isna().sum().to_numpy().nonzero()

In [ ]:
features_at_percent[20].keys()[nans]

In [ ]:
features_at_percent[20]
print(len(features_at_percent[100]))

In [ ]:
plt.plot(features_at_percent[10]['ewma_best_objective'])

In [ ]:
np.max([len(features_at_percent[199][features_at_percent[199]['solved_within_time_limit'] == False]) for i in range(1, 199)])

# Data Analysis
## Train Test Split

In [ ]:
def preprocessing(df):
    df1=df.drop(['mzn','dzn'], axis=1)
    df1.drop(df1.columns[df1.nunique() == 1], axis=1, inplace=True) #drop cols with constant value
    #rescale data
    transformer = MaxAbsScaler().fit(df1)
    df1 = pd.DataFrame(transformer.transform(df1), columns=df1.columns, index=df1.index) #normalise data
    return df1

In [ ]:
def create_split(df, test_size=0.25, random_state=22):
    return train_test_split(df.drop(columns = ["solved_within_time_limit"]),\
                                df["solved_within_time_limit"], test_size=0.25, random_state=22)

In [ ]:
df=features_at_percent[5] #THE NUMBER HERE IS THE % OF TL
df=preprocessing(df)
# training-testing split
X_train, X_test, y_train, y_test  = train_test_split(df.drop(columns = ["solved_within_time_limit"]),\
                                                     df["solved_within_time_limit"], test_size=0.25, random_state=23)

In [ ]:
y_test.value_counts()

In [ ]:
y_train.value_counts()

## Models

In [ ]:
models = {}

# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['LR'] = LogisticRegression(max_iter=1000, C=1000 , class_weight = 'balanced',random_state=22)

#Support Vector Machine
from sklearn.svm import SVC
models['SVM'] = SVC(kernel = 'rbf', class_weight = 'balanced', probability = True, random_state=22)

# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['RF'] = RandomForestClassifier(min_samples_leaf = 5, class_weight = 'balanced_subsample',random_state=22)

#Extra Tree
from sklearn.ensemble import ExtraTreesClassifier
models['ET'] = ExtraTreesClassifier(class_weight = 'balanced', random_state=22)

#Multi-layered perceptron
from sklearn.neural_network import MLPClassifier
models['MLP'] = MLPClassifier(random_state=22)

# Naive Bayes
#from sklearn.naive_bayes import GaussianNB
#models['NB'] = GaussianNB()

# Adaboost
#from sklearn.ensemble import AdaBoostClassifier
#models['AB'] = AdaBoostClassifier()

#KNN
#from sklearn.neighbors import KNeighborsClassifier
#models['KNN'] = KNeighborsClassifier(weights = 'distance')

# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models['DT'] = DecisionTreeClassifier(max_depth = 5, class_weight = 'balanced', random_state=22)

#Dummy classifier
from sklearn.dummy import DummyClassifier
models['DUM'] = DummyClassifier(strategy="most_frequent")

## Predicitions and Evaluation

In [ ]:
rf_reg = RandomForestClassifier(min_samples_leaf = 5)
rf_reg.fit(X_train,y_train)

predictions = rf_reg.predict(X_test)
importances = rf_reg.feature_importances_

print('The F1 score with all features of a RandomForestClassifier with min_samples_leaf of 5 is ', f1_score(y_test, predictions))


## All features

In [ ]:
print(X_train.columns)

In [ ]:

sorted_importance_indices = np.argsort(importances)[::-1]

plt.title('Feature Importance of Random Forest Regressor')
plt.bar(range(len(sorted_importance_indices)), importances[sorted_importance_indices], align='center')
plt.xticks(range(len(sorted_importance_indices)), X_train.columns[sorted_importance_indices], rotation=90)
plt.rcParams["figure.figsize"] = (20,3)
plt.show()

## Top 8 features

In [ ]:


sorted_importance_indices = np.argsort(importances)[::-1][:8]

plt.title('Feature Importance of Random Forest Regressor')
plt.bar(range(8), importances[sorted_importance_indices], align='center')
plt.xticks(range(8), X_train.columns[sorted_importance_indices], rotation=90)
plt.rcParams["figure.figsize"] = (20,3)
plt.show()

In [ ]:
sorted_importance_indices = np.argsort(importances)[::-1][:15]

def f1_with_n_top_features(n, importance_column_indices):
    summed = 0
    for i in range(0, 10):
        X_train, X_test, y_train, y_test  = train_test_split(df.drop(columns = ["solved_within_time_limit"]),\
                                                     df["solved_within_time_limit"], test_size=0.25)
        rf_reg = RandomForestClassifier(min_samples_leaf = 5)
        rf_reg.fit(X_train.iloc[:, importance_column_indices[:n]], y_train)

        predictions = rf_reg.predict(X_test.iloc[:, importance_column_indices[:n]])
        summed += f1_score(y_test, predictions)
    return summed / 10



f1_scores = [(i, f1_with_n_top_features(i, sorted_importance_indices)) for i in range(1, len(sorted_importance_indices) + 1)[::-1]]

plt.title(f'F1 score for top N features ({"OPTIMIZATION" if data_types_to_consider is InstanceType.OPTIMIZATION else "SAT"})')
plt.plot([a[0] for a in f1_scores], [a[1] for a in f1_scores])
plt.gca().set_ylim([0,1])
plt.rcParams["figure.figsize"] = (20,3)
plt.show()

print(f1_scores)



In [ ]:
def results_plt(X_train, y_train, X_test, y_test, perc_TL):
    train_accuracy, accuracy, precision, recall, auc, f1 = {}, {}, {}, {}, {}, {}
    figs, axs = plt.subplots(2,3,figsize=(20, 12))

    for i, key in enumerate(models.keys()):

        # Fit the classifier model
        models[key].fit(X_train, y_train)

        predictions = models[key].predict(X_test)
        predictions_prob = models[key].predict_proba(X_test)[:,1]
        train_predictions = models[key].predict(X_train)
        # Predic  
        # Calculate Accuracy, Precision and Recall Metrics
        accuracy[key] = accuracy_score(predictions, y_test)
        precision[key] = precision_score(predictions, y_test, zero_division=1)
        recall[key] = recall_score(predictions, y_test, zero_division=1)
        auc[key] = roc_auc_score(y_test, predictions_prob)
        train_accuracy[key] = accuracy_score(train_predictions, y_train)
        f1[key] = f1_score(y_test,predictions)
        #should it be (true, pred)? yes

        #To Display
        RocCurveDisplay.from_predictions(y_test, predictions_prob, name=key , ax=axs[0,0])
        axs[0,1].bar(key, accuracy[key]) 
        axs[0,2].bar(key, train_accuracy[key]) 
        axs[1,0].bar(key, recall[key])
        axs[1,1].bar(key, precision[key])
        axs[1,2].bar(key, f1[key])

        axs[0,0].set_title("ROC Curve for "+perc_TL+"% TL")
#         axs[0].set_ylabel("Accuracy")

        axs[0,1].set_title("Test Accuracy for "+perc_TL+"% TL")
        axs[0,1].set_ylabel("Accuracy")
        axs[0,1].grid(axis='y', color='gray', linestyle='dashed')

        axs[0,2].set_title("Train Accuracy for "+perc_TL+"% TL")
        axs[0,2].set_ylabel("Accuracy")
        axs[0,2].grid(axis='y', color='gray', linestyle='dashed')

        axs[1,0].set_title("Recall for "+perc_TL+"% TL")
        axs[1,0].set_ylabel("Recall")
        axs[1,0].grid(axis='y', color='gray', linestyle='dashed')


        axs[1,1].set_title("Precision "+perc_TL+"% TL")
        axs[1,1].set_ylabel("F1")
        axs[1,1].grid(axis='y', color='gray', linestyle='dashed')

        axs[1,2].set_title("F1 "+perc_TL+"% TL")
        axs[1,2].set_ylabel("F1")
        axs[1,2].grid(axis='y', color='gray', linestyle='dashed')
        
        
        [axs[y, x].tick_params(axis='x', labelrotation=60) for y in range(len(axs)) for x in range(len(axs[y]))]

In [ ]:
#Bar plot results  for testing and training using top 5 created features

results_plt(X_train, y_train, X_test, y_test, "5")

In [ ]:
def get_f1_scores(args):
    features_at_percent, percent, rep = args
    df_at_percent = features_at_percent[percent]
    df_at_percent = preprocessing(df_at_percent)
    
    try:
        X_train, X_test, y_train, y_test  = create_split(df_at_percent, random_state=percent + (100 * rep))
        model = RandomForestClassifier(min_samples_leaf = 5, class_weight = 'balanced_subsample',random_state=percent + (100 * rep))
        model.fit(X_train,y_train)
        predictions = model.predict(X_test)
        return f1_score(y_test, predictions)        
    except KeyError as e:
        return 0

In [ ]:
def get_f1_over_time_for_model(features_at_percent, repetitions=10):
    f1_scores_mean = []
    f1_scores_std = []
    for percent in tqdm(range(1, 200), desc="Percent"):
        with mp.Pool(mp.cpu_count()) as pool:
            f1_scores = pool.map(get_f1_scores, [(features_at_percent, percent, rep) for rep in range(repetitions)])
            
#             f1_scores = [get_f1_scores((features_at_percent, percent, rep)) for rep in range(repetitions)]

            f1_scores_mean.append(np.mean(f1_scores))
            f1_scores_std.append(np.std(f1_scores))
        
    return f1_scores_mean, f1_scores_std
    

In [ ]:
f1_scores_mean, f1_scores_std = get_f1_over_time_for_model(filtered_features_at_percent)

In [ ]:
plt.ylim((0, 1))
y = np.array(f1_scores_mean)
err = np.array(f1_scores_std)

plt.rcParams["figure.figsize"] = (10,6)
plt.plot(np.arange(0, 99.5, 0.5), y)
plt.title("F1 score at varying percentages of TL")
plt.fill_between(np.arange(0, 99.5, 0.5), y - err, y + err, alpha=0.4)
plt.tight_layout()
plt.savefig("f1_over_time.svg")

In [ ]:
problems_available_at_percentage = [len(filtered_features_at_percent[i]) for i in range(1, 200)]
plt.rcParams["figure.figsize"] = (5,4)
# plt.title("Problems unsolved after %")
plt.xlabel("Percentage of TL")
plt.ylabel("Prob. Unsolved")
plt.plot(np.arange(0, 99.5, 0.5), problems_available_at_percentage)
plt.tight_layout()
# plt.vlines([5], ymin=0, ymax=problems_available_at_percentage[10], color='black', alpha=0.5)
plt.savefig("problems_avail_at_perc.svg")

## Regression

In [ ]:
all_data_reg = all_data[all_data.time_to_solution <  7199* 1000] #<2h

In [ ]:
all_data_reg

In [ ]:
features_at_percent_reg = {}
for i in range (1,100):
    df_i_reg=[]
    for id, problem in all_data_reg.iterrows():
            for index, p in enumerate(problem.statistics):
                if index == i:
                    new_p = dict(p['features'])
                    new_p=cleanup(new_p)
                    if i!=1:
                        new_p=gradients(df_prev.loc[id], new_p)
                    new_p['mzn'] = problem['mzn']
                    new_p['dzn'] = problem['dzn']
                    new_p['time_to_solution'] = problem['time_to_solution']
                    df_i_reg.append((id, new_p))
    df_i_reg = pd.DataFrame([a[1] for a in df_i_reg], index=[a[0] for a in df_i_reg])
    df_i_reg=df_i_reg.fillna(value = 0)
    if i!=0:   
        features_at_percent_reg[i]=df_i_reg
    df_prev=df_i_reg

In [ ]:
features_at_percent[5]

In [ ]:
grid=[5,10,20,50,90]
for i in grid:
    print("At",i,"%:")
    print(len(features_at_percent[i].index))
    print(len(features_at_percent_reg[i].index))
    

In [ ]:
df_reg=features_at_percent_reg[1] #THE NUMBER HERE IS THE % OF TL
df_reg=preprocessing(df_reg)
# training-testing split
X_train_reg, X_test_reg, y_train_reg, y_test_reg  = train_test_split(df_reg.drop(columns = ["time_to_solution"]),\
                                                     df_reg["time_to_solution"], test_size=0.25, random_state=22)

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=1000000000000)
regr.fit(X_train_reg, y_train_reg)
predictions=regr.predict(X_test_reg)
sk_score = regr.score(X_test_reg, y_test_reg)
sk_score

In [ ]:
#SUPPORT VECTOR MACHINE
from sklearn import svm
regr = svm.SVR()
regr.fit(X_train_reg, y_train_reg)
predictions=regr.predict(X_test_reg)
sk_score = regr.score(X_test_reg, y_test_reg)
sk_score

In [ ]:
#RIDGE
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(X_train_reg, y_train_reg)
predictions=clf.predict(X_test_reg)
sk_score = clf.score(X_test_reg, y_test_reg)
sk_score

In [ ]:
#LASSO
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train_reg, y_train_reg)
predictions=clf.predict(X_test_reg)
sk_score = clf.score(X_test_reg, y_test_reg)
sk_score

In [ ]:
#ELASTICNET
from sklearn.linear_model import ElasticNet
regr = ElasticNet()
regr.fit(X_train_reg, y_train_reg)
predictions=regr.predict(X_test_reg)
sk_score = regr.score(X_test_reg, y_test_reg)
sk_score

In [ ]:
#TREE
from sklearn import tree
clf = tree.DecisionTreeRegressor(max_depth=5)
clf.fit(X_train_reg, y_train_reg)
predictions=clf.predict(X_test_reg)
sk_score = clf.score(X_test_reg, y_test_reg)
sk_score

In [ ]:
#PLS
from sklearn.cross_decomposition import PLSRegression
pls2 = PLSRegression(n_components=2)
pls2.fit(X_train_reg, y_train_reg)
predictions=pls2.predict(X_test_reg)
sk_score = pls2.score(X_test_reg, y_test_reg)
sk_score

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel).fit(X_train_reg, y_train_reg)
predictions=gpr.predict(X_test_reg)
sk_score = gpr.score(X_test_reg, y_test_reg)
sk_score